In [2]:
import Convex
import Mosek
import JuMP
import HD

Generate data

In [15]:
n = 2000
p = 5000
s = 10

rho = 0.5
covMat = rho.^abs([1:p]*ones(p)' - ones(p)*[1:p]')

X = randn(n, p) * sqrtm(covMat)
true_beta = vcat( (1.+rand(s)) .* (2.*rand(0:1,s).-1), zeros(p-s))
Y = X * true_beta + 0.1 .* randn(n)

@show true_beta[1:s];

true_beta[1:s] => [-1.9717088460445666,1.6029695699517774,1.2136801528987835,1.8519810438343187,1.7917513822211246,-1.9795642489251102,1.9248521815171311,-1.912970420715021,-1.0687110015839774,-1.9852347946888242]


Functions for quantile regression

In [3]:
# implementation using convex
function qr(X, Y, lambda)
    
    solver = Mosek.MosekSolver(LOG=0)
    Convex.set_default_solver(solver);

    n, p = size(X)
    theta = Convex.Variable(p);
    prob = Convex.minimize(sum(abs(Y-X*theta)) / n / 2 + lambda * norm(theta, 1))
    Convex.solve!(prob)

    theta.value
end

#implementation using JuMP
function qr_primal(X, Y, lambda, tau)
    solverJ = JuMP.Model(solver=Mosek.MosekSolver(LOG=1))
    n, p = size(X)

    oneN = ones(n)
    @JuMP.defVar(solverJ, z[1:p])
    @JuMP.defVar(solverJ, t[1:p])
    @JuMP.defVar(solverJ, u[1:n])
    @JuMP.defVar(solverJ, v[1:n])
    @JuMP.setObjective(solverJ, Min, (tau*dot(oneN, u) + (1-tau)*dot(oneN, v)) / n + dot(lambda, t))
    for i=1:n
        @JuMP.addConstraint(solverJ, Y[i] - dot(vec(X[i,:]), z) == u[i] - v[i])
        @JuMP.addConstraint(solverJ, u[i] >= 0)
        @JuMP.addConstraint(solverJ, v[i] >= 0)
    end
    for i=1:p
        @JuMP.addConstraint(solverJ, z[i] <= t[i])
        @JuMP.addConstraint(solverJ, -z[i] <= t[i])
    end

    JuMP.solve(solverJ)

    JuMP.getValue(z)
end

#implementation unsing JuMP dual
function qr_dual(X, Y, lambda, tau)
    solverJ = JuMP.Model(solver=Mosek.MosekSolver(LOG=1))
    n, p = size(X)

    @JuMP.defVar(solverJ, xi[1:n])
    @JuMP.defVar(solverJ, cp[1:p])
    @JuMP.defVar(solverJ, cn[1:p])
    @JuMP.setObjective(solverJ, Max, -dot(xi, Y))
    for i=1:n
        @JuMP.addConstraint(solverJ, xi[i] <= 1-tau)
        @JuMP.addConstraint(solverJ, xi[i] >= -tau)    
    end
    for i=1:p
        @JuMP.addConstraint(solverJ, dot(vec(X[:, i]), xi) == cp[i] - cn[i])
        @JuMP.addConstraint(solverJ, cp[i] >= 0)
        @JuMP.addConstraint(solverJ, cn[i] >= 0)
        @JuMP.addConstraint(solverJ, cp[i] + cn[i] <= lambda[i])
    end

    JuMP.solve(solverJ)

    JuMP.getValue(xi)
end


qr_dual (generic function with 1 method)

In [6]:
?mapslices

INFO: Loading help data...


Base.mapslices(f, A, dims)

   Transform the given dimensions of array "A" using function "f".
   "f" is called on each slice of "A" of the form
   "A[...,:,...,:,...]". "dims" is an integer vector specifying
   where the colons go in this expression. The results are
   concatenated along the remaining dimensions. For example, if
   "dims" is "[1,2]" and A is 4-dimensional, "f" is called on
   "A[:,:,i,j]" for all "i" and "j".


In [9]:
A = [1 2 3 4; 1 2 3 4]

2x4 Array{Int64,2}:
 1  2  3  4
 1  2  3  4

In [8]:
mapslices(norm, A, 1)

1x3 Array{Float64,2}:
 1.41421  2.82843  4.24264